In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
# np.load('./y/g1dual1_y.npy').shape

In [3]:
# path = './y/'
# label = []
# for filename in sorted(os.listdir(path)):
#     f = os.path.join(path, filename)
#     y = np.load(f)
#     print(y.shape)
#     label = np.hstack((label, y))

# print(label.shape)

In [4]:
# path = './data/'
# filename = sorted(os.listdir(path))[0]
# f = os.path.join(path, filename)
# data = np.load(f)
# print(data.shape)

# for filename in sorted(os.listdir(path)):
#     if filename == 'g1dual1_X.npy':
#         print(filename)
#         continue
#     f = os.path.join(path, filename)
#     x = np.load(f)
#     print(x.shape)
#     data = np.concatenate((data, x), axis=0)

# # print(data.shape)

In [5]:
# data.shape

In [6]:
# np.save('./data/data.npy', data)
# np.save('./data/label.npy', label)

In [7]:
X = np.load('./data/data.npy')
y = np.load('./data/label.npy')

print(X.shape)
print(y.shape)

(7164, 32, 410)
(7164,)


In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from mne.preprocessing import Xdawn
from pyriemann.estimation import ERPCovariances
from collections import OrderedDict
from pyriemann.classification import MDM
from pyriemann.estimation import XdawnCovariances
from sklearn.metrics import classification_report
from mne.decoding import Vectorizer
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

lda = LDA(shrinkage='auto', solver='eigen') #Regularized LDA
svc = SVC(kernel = 'rbf')

clfs = OrderedDict()

n_components = 3

clfs['ERP + MDM'] = make_pipeline(ERPCovariances(estimator='oas'), MDM())
clfs['Xdawn + RegLDA'] = make_pipeline(XdawnCovariances(n_components, 
                            estimator='oas'), Vectorizer(), lda)
clfs['Xdawn + MDM'] = make_pipeline(XdawnCovariances(n_components,
                            estimator='oas'), MDM())
clfs['Xdawn + SVM'] = make_pipeline(XdawnCovariances(n_components, estimator = 'oas'), Vectorizer(), svc)

print(clfs)
# Cross validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)

for clf in clfs:
    # Do cross-validation
    preds = np.empty(len(y))
    for train, test in cv.split(X, y):  #Xdawn takes in epoch object
        print(X.shape, y.shape)
        clfs[clf].fit(X[train], y[train])
        preds[test] = clfs[clf].predict(X[test])

    # Classification report
    target_names = ['Non-target', 'targets']
    report = classification_report(y, preds, target_names=target_names)
    print(clf)
    print(report)
    
# measure auc
auc = []
methods = []
for m in clfs:
    res = cross_val_score(clfs[m], X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    auc.extend(res)
    methods.extend([m]*len(res))
    
results = pd.DataFrame(data=auc, columns=['AUC'])
results['Method'] = methods

OrderedDict([('ERP + MDM', Pipeline(steps=[('erpcovariances', ERPCovariances(estimator='oas')),
                ('mdm', MDM())])), ('Xdawn + RegLDA', Pipeline(steps=[('xdawncovariances',
                 XdawnCovariances(estimator='oas', nfilter=3)),
                ('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x7f4aefb3baf0>),
                ('lineardiscriminantanalysis',
                 LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen'))])), ('Xdawn + MDM', Pipeline(steps=[('xdawncovariances',
                 XdawnCovariances(estimator='oas', nfilter=3)),
                ('mdm', MDM())])), ('Xdawn + SVM', Pipeline(steps=[('xdawncovariances',
                 XdawnCovariances(estimator='oas', nfilter=3)),
                ('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x7f4ab7e34f40>),
                ('svc', SVC())]))])
(7164, 32, 410) (7164,)
(7164, 32, 410) (7164,)
(7164, 32, 410) (7164,)
(7164, 32, 410) 

In [9]:
from sklearn.model_selection import StratifiedKFold
from mne.preprocessing import Xdawn
from pyriemann.estimation import XdawnCovariances
from sklearn.metrics import classification_report
from mne.decoding import Vectorizer
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

svc = SVC(kernel = 'rbf')
n_components = 3

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)
model = make_pipeline(XdawnCovariances(n_components, estimator='oas'), Vectorizer(), svc)

preds = np.empty(len(y))
for train, test in cv.split(X, y):  #Xdawn takes in epoch object
    model.fit(X[train], y[train])
    preds[test] = model.predict(X[test])
    # print("preds.shape", preds.shape)
    
# Classification report
target_names = ['Non-target', 'targets']
print("y.shape", y.shape)
print("---SVM---")
report = classification_report(y, preds, target_names=target_names)
print(report)

X.shape:  (7164, 32, 410)
y.shape:  (7164,)
y.shape (7164,)
---SVM---
              precision    recall  f1-score   support

  Non-target       0.83      1.00      0.91      5970
     targets       0.75      0.00      0.01      1194

    accuracy                           0.83      7164
   macro avg       0.79      0.50      0.46      7164
weighted avg       0.82      0.83      0.76      7164



In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from mne.preprocessing import Xdawn
from pyriemann.estimation import XdawnCovariances
from sklearn.metrics import classification_report
from mne.decoding import Vectorizer
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline

lda = LDA(shrinkage='auto', solver='eigen') #Regularized LDA

print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

n_components = 3

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)
model = make_pipeline(XdawnCovariances(n_components, estimator='oas'), Vectorizer(), lda)

preds = np.empty(len(y))
for train, test in cv.split(X, y):  #Xdawn takes in epoch object
    model.fit(X[train], y[train])
    preds[test] = model.predict(X[test])
    # print("preds.shape", preds.shape)
    
# Classification report
target_names = ['Non-target', 'targets']
print("y.shape", y.shape)
print("---Regularized LDA---")
report = classification_report(y, preds, target_names=target_names)
print(report)

X.shape:  (7164, 32, 410)
y.shape:  (7164,)
y.shape (7164,)
---Regularized LDA---
              precision    recall  f1-score   support

  Non-target       0.88      0.96      0.92      5970
     targets       0.64      0.31      0.42      1194

    accuracy                           0.86      7164
   macro avg       0.76      0.64      0.67      7164
weighted avg       0.84      0.86      0.83      7164

